In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
# create helper functions
# weight function
def init_weight(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [11]:
# bias function

def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape = shape)
    return tf.Variable(init_bias_vals)

In [13]:
# conv2D
def conv2d(x,W):
    # x --> input tensor where [batch,H(height),W(width),Channel(gray or RGB )]
    # W --> kernel where [ Filter H, Filter W , Channels In , Channels Out]
    return tf.nn.conv2d(x,W,strides =[1,1,1,1],padding = "SAME")

    

In [15]:
#pooling convenient function

def max_pool_2by2(x):
    # x input tensor
    # we want to pull from the heights and width of the input [Batch, H, W, Channel] therefore ksize -- [1,2,2,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1] , strides=[1,2,2,1] , padding= "SAME")

In [22]:
# convulutional layer
def convolutional_layer(input_x, shape):
    W = init_weight(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [17]:
# normal fully connected layer
def normal_full_layer(input_layer , size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weight([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

In [18]:
x = tf.placeholder(tf.float32,shape = [None , 784])

In [20]:
y_true =tf.placeholder(tf.float32,shape= [None , 10])##one hot encoded labels

In [21]:
#layers
x_image = tf.reshape(x,[-1,28,28,1])##1 is the channel which is gray scaled for this example

In [24]:
#convolutional layers
convo_1 = convolutional_layer(x_image , shape= [5,5,1,32])##32 feathres for each 5,5 batch
convo_1_pooling = max_pool_2by2(convo_1)

In [26]:
convo_2 = convolutional_layer(convo_1_pooling,shape = [5,5,32,64])
convo_2_pooling =max_pool_2by2(convo_2)

In [29]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [30]:
# drop out
hold_prob = tf.placeholder(tf.float32)
full_one_dropout =tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [31]:
y_pred = normal_full_layer(full_one_dropout,10)

In [33]:
#loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true,
                                                                       logits = y_pred))

In [34]:
#optimizer
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(cross_entropy)

In [38]:
init = tf.global_variables_initializer()

In [40]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x , batch_y  = mnist.train.next_batch(50)
        sess.run(train, feed_dict= {x:batch_x , y_true: batch_y , hold_prob : 0.5 })
        
        if i%100 ==0:
            print("On Step :{}".format(i))
            print("Accuracy: ")
            
            matches =tf.equal(tf.argmax(y_pred , 1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict = {x:mnist.test.images, y_true : mnist.test.labels, hold_prob:1.0}))
            print("\n")

On Step :0
Accuracy: 
0.1308


On Step :100
Accuracy: 
0.9494


On Step :200
Accuracy: 
0.9628


On Step :300
Accuracy: 
0.9665


On Step :400
Accuracy: 
0.9736


On Step :500
Accuracy: 
0.9792


On Step :600
Accuracy: 
0.9748


On Step :700
Accuracy: 
0.9844


On Step :800
Accuracy: 
0.9785


On Step :900
Accuracy: 
0.9821


On Step :1000
Accuracy: 
0.9851


On Step :1100
Accuracy: 
0.9847


On Step :1200
Accuracy: 
0.9871


On Step :1300
Accuracy: 
0.9862


On Step :1400
Accuracy: 
0.986


On Step :1500
Accuracy: 
0.9881


On Step :1600
Accuracy: 
0.9873


On Step :1700
Accuracy: 
0.986


On Step :1800
Accuracy: 
0.9882


On Step :1900
Accuracy: 
0.9854


On Step :2000
Accuracy: 
0.9885


On Step :2100
Accuracy: 
0.9881


On Step :2200
Accuracy: 
0.9874


On Step :2300
Accuracy: 
0.9897


On Step :2400
Accuracy: 
0.9892


On Step :2500
Accuracy: 
0.9881


On Step :2600
Accuracy: 
0.9888


On Step :2700
Accuracy: 
0.9881


On Step :2800
Accuracy: 
0.9884


On Step :2900
Accuracy: 
0.9